In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import lr_scheduler
import time 
import tqdm
from PIL import Image
train_on_gpu = True
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR

import cv2
import albumentations
from albumentations import torch as AT

In [ ]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
# !pip install matplotlib
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from PIL import Image
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
test = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')
sample_submission = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
print("Dataset is now loaded")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
sample_submission.head()

**0 - No DR

**1 - Mild

**2 - Moderate
**
3 - Severe
**

4 - Proliferative DR******

In [ ]:
train['diagnosis'].value_counts().plot(kind='bar');
plt.title("Class")

# Let's See some Retinas and Check what really Happens at the Molecular Level when patients suffer from diabetic Retinotherapy

import torch
from torchvision import models, transforms

# densenet121
model = models.densenet121()
model.load_state_dict(torch.load('densenet121.pth'))

# densenet201
model = models.densenet201()
model.load_state_dict(torch.load('densenet201.pth'))

# resnet50
model = models.resnet50()
model.load_state_dict(torch.load('resnet50.pth'))

# resnet34
model = models.resnet34()
model.load_state_dict(torch.load('resnet34.pth'))


In [ ]:
import torch
from torchvision import models, transforms

# densenet121
model = models.densenet121()
model.load_state_dict(torch.load('densenet121.pth'))

# densenet201
model = models.densenet201()
model.load_state_dict(torch.load('densenet201.pth'))

# resnet50
model = models.resnet50()
model.load_state_dict(torch.load('resnet50.pth'))

# resnet34
model = models.resnet34()
model.load_state_dict(torch.load('resnet34.pth'))


In [ ]:
fig = plt.figure(figsize=(60,40))
# Display 10 images from each class 
for class_id in sorted(train['diagnosis'].unique()):
    for i, (idx, row) in enumerate(train.loc[train['diagnosis'] == class_id].sample(10).iterrows()):
        ax = fig.add_subplot(5, 10, class_id * 10 + i + 1, xticks=[], yticks=[])
        im = Image.open(f"../input/aptos2019-blindness-detection/train_images/{row['id_code']}.png")
        #code to move to next frame

        print("These are ",plt.imshow(im))
        img = np.array(im)
        print(img.shape)
#         print(im.shape)
        ax.set_title(f'Label: {class_id}')
# try:
#     im.shape
#     print("checked for shape".format(img.shape))
#     except AttributeError:
# print("shape not found")


**Personally I see little differences between images**

# Data preparation
Preprocessing the Data Applying One Hot Encoding and Label Encoders

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# Declaring a Function to that Task
# We are using 
def perparelabels(y):
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

    y = onehot_encoded
    return y, label_encoder

In [ ]:
y, le = perparelabels(train['diagnosis'])


In [ ]:
class GlassDataset(Dataset):
    def __init__(self, df, datatype='train', transform = transforms.Compose([transforms.CenterCrop(32),transforms.ToTensor()]), y = None):
        self.df = df
        self.datatype = datatype
        self.image_files_list = [f'../input/aptos2019-blindness-detection/{self.datatype}_images/{i}.png' for i in df['id_code'].values]
        if self.datatype == 'train':
            self.labels = y
        else:
            self.labels = np.zeros((df.shape[0], 5))
        self.transform = transform

    def __len__(self):
        return len(self.image_files_list)

    def __getitem__(self, idx):
        img_name = self.image_files_list[idx]
        img = cv2.imread(img_name)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        image = self.transform(image=img)
        image = image['image']

        img_name_short = self.image_files_list[idx].split('.')[0]

        label = self.labels[idx]
        if self.datatype == 'test':
            return image, label, img_name
        else:
            return image, label


In [ ]:
data_transforms = albumentations.Compose([
    albumentations.Resize(224, 224),
    albumentations.HorizontalFlip(),
    albumentations.RandomBrightness(),
    albumentations.ShiftScaleRotate(rotate_limit=15, scale_limit=0.10),
    albumentations.JpegCompression(80),
    albumentations.HueSaturationValue(),
    albumentations.Normalize(),
    AT.ToTensor()
    ])
data_transforms_test = albumentations.Compose([
    albumentations.Resize(224, 224),
    albumentations.Normalize(),
    AT.ToTensor()
    ])

dataset = GlassDataset(df=train, datatype='train', transform=data_transforms, y=y)
test_set = GlassDataset(df=test, datatype='test', transform=data_transforms_test)
tr, val = train_test_split(train.diagnosis, stratify=train.diagnosis, test_size=0.1)
train_sampler = SubsetRandomSampler(list(tr.index))
valid_sampler = SubsetRandomSampler(list(val.index))
batch_size = 64
num_workers = 0
# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=num_workers)


In [ ]:
test_set = GlassDataset(df=test, datatype='test', transform=data_transforms_test)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=num_workers)

In [ ]:
# !pip install kaggle
model_conv = torchvision.models.resnet50()
# !kaggle download -c https://storage.googleapis.com/kaggle-datasets/107131/255419/pytorch-pretrained-image-models.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1567062005&Signature=ArQBmqltk6F71KVNZDHe%2F0q%2FG2KJTEejKWjC6Sb9j%2FFcto8LQCaxn20ZBx7H9j0ocJWX3%2BrHRISFx65O%2B2Ma52HMTyxbtf2%2FnrKwsjYuDGPbFyqj%2Bbf0RWmcu5bQJYXy0wfpzUCsZQEO0ja7%2Bjv0%2BrIEQVz7QNBbMBzK5TDXzDl6jdjhidmFRiIC%2FjSQgTQNymKLk8wfXLFAXZsWDpNWYk%2BJhieQX4uNjixkK2rDlmKJK0cXzrUIBOGY8aUUH6Nfs%2BVZK89jk8e3zYVG6qGEz67NFRxBYOkP9Iwq%2B6mOQzOviJv%2BReVK88nWvuThbwBZ3XgjrIfYSCudRP0WhdRhCA%3D%3D

model_conv.load_state_dict(torch.load('../input/pytorch-pretrained-image-models/resnet50.pth'))

In [ ]:
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(2048, 5)


In [ ]:
# libraries
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import lr_scheduler
import time 
import tqdm
from PIL import Image
train_on_gpu = True
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR

import cv2
import albumentations
from albumentations import torch as AT
import scipy.optimize as optimize

valid_loss_min = np.Inf
patience = 5
# current number of epochs, where validation loss didn't increase
p = 0
# whether training should be stopped
stop = False

# number of epochs to train the model
n_epochs = 20
for epoch in range(1, n_epochs+1):
    print(time.ctime(), 'Epoch:', epoch)

    train_loss = []
    train_auc = []

    for batch_i, (data, target) in enumerate(train_loader):

        data, target = data.cuda(), target.cuda()

        optimizer.zero_grad()
        output = model_conv(data)
        loss = criterion(output, target.float())
        train_loss.append(loss.item())
        
        a = target.data.cpu().numpy()
        b = output[:,-1].detach().cpu().numpy()
        # train_auc.append(roc_auc_score(a, b))
        loss.backward()
        optimizer.step()
    
    model_conv.eval()
    val_loss = []
    val_auc = []
    for batch_i, (data, target) in enumerate(valid_loader):
        data, target = data.cuda(), target.cuda()
        output = model_conv(data)

        loss = criterion(output, target.float())

        val_loss.append(loss.item()) 
        a = target.data.cpu().numpy()
        b = output[:,-1].detach().cpu().numpy()
        # val_auc.append(roc_auc_score(a, b))

    # print(f'Epoch {epoch}, train loss: {np.mean(train_loss):.4f}, valid loss: {np.mean(val_loss):.4f}, train auc: {np.mean(train_auc):.4f}, valid auc: {np.mean(val_auc):.4f}')
    print(f'Epoch {epoch}, train loss: {np.mean(train_loss):.4f}, valid loss: {np.mean(val_loss):.4f}.')
    
    valid_loss = np.mean(val_loss)
    scheduler.step(valid_loss)
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model_conv.state_dict(), 'model.pt')
        valid_loss_min = valid_loss
        p = 0

    # check if validation loss didn't improve
    if valid_loss > valid_loss_min:
        p += 1
        print(f'{p} epochs of increasing val loss')
        if p > patience:
            print('Stopping training')
            stop = True
            break        
            
    if stop:
        break
